# DESY depth profiling data analysis workbook [2021]

This is a workbook used to integrate synchrotron data from DESY then generate the bg-spline and peak-index files required for running CMWP.

## Import stuff

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib qt
import os
import globs
import pyFAI, pyFAI.azimuthalIntegrator
from pyFAI.multi_geometry import MultiGeometry
import csv
from scipy.interpolate import CubicSpline
from shutil import copyfile
from cmwp_tools import load_tifs, getHexReflections

## Define settings

In [78]:
### Experiment specific settings

base = "/mnt/manchester_rds/202107_DESY_DP/"                            # This is the base directory where the data is stored

calib_files = [base + 'det1_calib.poni',
               base + 'det2_calib.poni',
               base + 'det3_calib.poni',
               base + 'det4_calib.poni']                                # These are the calibration files (.poni) for the detectors

mask_files = [base + 'det1_mask.edf',
               base + 'det2_mask.edf',
               base + 'det3_mask.edf',
               base + 'det4_mask.edf']                                  # These are the mask files for each detector

templates = base + "templates/"                                         # This is where the template files are stored. These are copied for each integration.

intpoints = 10000                                                       # This is how many integration points to use for pyFAI

wavelength = 0.1836803                                                  # Wavelength in angstrom
a=3.232                                                                 # a lattice paramater in angstrom
c=5.147                                                                 # c lattice paramater in angstrom

baseline=[3.45, 4.6, 5.1, 6.2, 6.8, 8.3, 9.3, 10.32, 11.45]             # What 2theta points to calculte the background spline from

limits=[3.4,11.5]                                                       # 2theta bounds of integration

searchrange = 100                                                       # How many data points each side of the approximate 2theta peak position to search for the true peak


In [79]:
### Sample specific settings
               
directory = base + "depth_prof/sam1a"                                   # Directory containing the data (should have folders 1, 2, 3, 4 and a .fio file within)

outputdir = "/home/rhys/Documents/CMWP-210315/DESY_202107/sam1a/"       # This is the output directory (normally a sub-folder in your CMWP dir)

## Read log file and make pandas table

The .fio file contains a table of motor positions (ie idtz2, idty1), image names and whether the image was a clearing frame or actual exposure. The section reads in the file into a Pandas dataframe for ease of use.

In [81]:
# Get .fio file path
names = glob.glob(directory+'/*.fio')
if len(names) == 1:
    fio_file = names[0]
else:
    raise Exception('Either there are no .fio files in the directory, or there are more than 1')
    
# Read in column names and data type
colnames = []; formats = [];
with open(fio_file) as input_data:
    for i, line in enumerate(input_data):
        if ' Col' in line:
            colnames.append(' '.join(line.split(' ')[3:-1]))
            skip = i+1
            if 'DOUBLE' in line.split(' ')[-1]: formats.append('f4')
            if 'INTEGER' in line.split(' ')[-1]: formats.append('i4')
            if 'STRING' in line.split(' ')[-1]: formats.append('str')

# Read in log file into dataframe and remove clearing frames
df = pd.read_csv(fio_file, names = colnames, skiprows=skip, sep=' ', skipinitialspace=True) 
df = df[df.type != 'clearing']

# Get list of unique motor position values
z_values = df['idtz2'].unique()
y_values = df['idty1(encoder)'].unique()
    
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)

if not os.path.exists(outputdir):
    os.makedirs(outputdir)
if not os.path.exists(outputdir + '/0plots'):
    os.makedirs(outputdir + '/0plots')


In [82]:
# Print the dataframe
df

,idty1(encoder),end pos,idtz2,channel,filename,type,unix time
0,-0.166467,-0.166467,12.583,1,sam1a_00002.cbf,exposure,1.625001e+09
1,0.166867,0.166867,12.583,1,sam1a_00003.cbf,exposure,1.625001e+09
2,0.500200,0.500200,12.583,1,sam1a_00004.cbf,exposure,1.625001e+09
3,0.166867,0.166867,12.585,1,sam1a_00006.cbf,exposure,1.625001e+09
4,-0.166467,-0.166467,12.585,1,sam1a_00007.cbf,exposure,1.625001e+09
...,...,...,...,...,...,...,...
148,-0.166467,-0.166467,12.681,1,sam1a_00199.cbf,exposure,1.625002e+09
149,-0.499800,-0.499800,12.681,1,sam1a_00200.cbf,exposure,1.625002e+09
150,-0.166467,-0.166467,12.683,1,sam1a_00202.cbf,exposure,1.625002e+09
151,0.166867,0.166867,12.683,1,sam1a_00203.cbf,exposure,1.625002e+09


## Integration, bg-spline and peak-index creation

This section takes all the images in the above table and integrates them according to the calibrations defined previously for all 4 detectors. 
This is saved as a .dat file with a prefix containing the motor positions. Then, the files in the template directory are copied with the same prefix. 
A background spline is created from the baseline points specified above and saved with the .bg-spline.dat suffix.
The a peak-index.dat file is made based on the Zr indexes specifed above.

In [96]:
peak_name, peak_pos = getReflections(crystalType='hcp', a=a, c=c, wavelength=wavelength, printReflections=False)

ais = [pyFAI.load(calib_file) for calib_file in calib_files]
mask = [load_tifs(mask_file) for mask_file in mask_files]
ais = MultiGeometry(ais, unit='2th_deg', radial_range=(limits[0], limits[1]))

print('z from {0:.3f} to {1:.3f}'.format(np.min(z_values), np.max(z_values)))
print('y from {0:.3f} to {1:.3f}'.format(np.min(y_values), np.max(y_values)))

int_list = []
for index, row in df.iterrows():
    y=row['idty1(encoder)']
    z=row['idtz2']
    
    prefix = 'y_{1:.3f}_z_{0:.3f}'.format(z,y)

    file = (df[(df['idtz2']==z) & (df['idty1(encoder)']==y)])['filename'].values[0]
    print('\rCurrent:   {0} / {1}\ty = {2:.3f} / {3:.3f}\tz = {4:.3f} / {5:.3f}     [{6}] '
          .format(index+1, len(df.index), y, np.max(y_values), z, np.max(z_values), file), end='')

    files = [directory + '/1/' + file, directory + '/2/' + file, directory + '/3/' + file, directory + '/4/' + file]
    
    data = [load_tifs(file) for file in files]

    output = ais.integrate1d(data, npt=intpoints, lst_mask=mask, correctSolidAngle=True, polarization_factor=0.99)

    xvals=output[0]; yvals=output[1];
    yvals = yvals / 100000
    #yvals = yvals - np.min(yvals)
    #yvals += 10000

    for templateName in glob.glob(templates + '*'):
        copyfile(templateName, outputdir + prefix + templateName.split('/')[-1][8:])

    ####################### Save integrated data ######################

    with open(outputdir + prefix + '.dat', 'w+') as f:
         np.savetxt(fname = f, X=np.transpose([xvals, yvals]), fmt = ('%1.5f'))

    ########################### Save figure ###########################

    plt.ioff()
    fig, (ax2) = plt.subplots(1, 1, figsize=(16,8))
    ax2.set_title('Integrated data');
    ax2.set_xlabel('2theta (deg)'); 
    ax2.set_ylabel('Intensity');
    ax2.plot(xvals, yvals)
    ax2.set_xlim(np.min(xvals)+0.01, np.max(xvals)-0.01)
    #ax2.set_ylim(np.min(yvals)-2, np.max(yvals)*1.1)

    x_plot_list = []
    y_plot_list = []

    ########################### Make bg-spline ###########################
    baseline_int = []
    for j in baseline:
        num_index=np.argmin(np.abs(xvals-j))

        baseline_int.append(np.mean(yvals[num_index-5:num_index+5]))

    baseline, baseline_int = (list(t) for t in zip(*sorted(zip(baseline, baseline_int))))

    with open(outputdir + prefix + '.bg-spline.dat', 'w+') as f:
        np.savetxt(fname = f, X=np.transpose([baseline, baseline_int]), fmt = ('%1.5f'))
        
    cs = CubicSpline(baseline, baseline_int)

    ax2.plot(xvals, cs(xvals))
    ax2.plot(baseline, baseline_int, 'o',c='r')

    ########################### Make peak-index ###########################

    if len(peak_pos) != len(peak_name):
        raise ValueError('peak_pos and peak_name arrays should be the same size')

    with open(outputdir + prefix + '.peak-index.dat', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        for name, pos in zip(peak_name, peak_pos):

            if np.min(xvals) < pos < np.max(xvals):         # if the peak is within the data
                
                # approx peak postion
                approx_peak_index = np.argmin(np.abs(xvals-pos))                    # get the index of the peak

                # get actual peak position
                peak_index = np.argmax(yvals[approx_peak_index-searchrange:approx_peak_index+searchrange])+approx_peak_index-searchrange
                yval = yvals[peak_index]

                # draw line and print name
                ax2.axvline(xvals[peak_index], alpha=0.1, c='r')
                ax2.text(xvals[peak_index], yval+10, name, horizontalalignment = 'center', c='r')

                intensity = yval - cs(xvals)[peak_index]

                writer.writerow(['{0:.4f} {1:.1f} {2} 0'.format(xvals[peak_index], intensity, name)])

    #Append to list
    int_list.append(np.sum(yvals - cs(xvals)))

    ## Save plots #################################

    plt.savefig(outputdir + '/0plots/plot_' + prefix + '.pdf')
    plt.yscale('log')
    plt.savefig(outputdir + '/0plots/log_' + prefix + '.pdf')
    plt.close()


z from 12.583 to 12.683
y from -0.500 to 0.500
Current:   153 / 153	y = 0.500 / 0.500	z = 12.683 / 12.683     [sam1a_00204.cbf]  

## Plot integrated intensities

In [84]:
for val in df['idty1(encoder)'].unique():
    yval = np.array(int_list)[np.array(df['idty1(encoder)'].tolist() == val)]
    zval = np.array(df['idtz2'].tolist())[np.array(df['idty1(encoder)'].tolist() == val)]
    
    # Take differential and calculate edge position
    edge = zval[np.argmax(np.gradient(yval))]
    
    plt.plot(zval, yval, label = 'y = {0:.3f}   edge = {1:.3f}'.format(val, edge))
    
plt.xlabel('Z position (mm)')
plt.ylabel('Integrated intensity')
plt.show()
plt.legend(loc='lower right')

plt.savefig(outputdir + '0integrated_intensity.pdf')

## Make a bash script

Save this text into a .sh file and run it - this will execute CMWP for each file sequentially

In [85]:
cmwpfolder = "/home/rhys/Documents/CMWP-210315/"

for index, row in df.iterrows():
    y=row['idty1(encoder)']; z=row['idtz2'];    
    
    print('./evaluate ' + outputdir.split(cmwpfolder)[-1] + 'y_{1:.3f}_z_{0:.3f}.dat auto'.format(z,y))

./evaluate DESY_202107/sam1a/y_-0.166_z_12.583.dat auto
./evaluate DESY_202107/sam1a/y_0.167_z_12.583.dat auto
./evaluate DESY_202107/sam1a/y_0.500_z_12.583.dat auto
./evaluate DESY_202107/sam1a/y_0.167_z_12.585.dat auto
./evaluate DESY_202107/sam1a/y_-0.166_z_12.585.dat auto
./evaluate DESY_202107/sam1a/y_-0.500_z_12.585.dat auto
./evaluate DESY_202107/sam1a/y_-0.166_z_12.587.dat auto
./evaluate DESY_202107/sam1a/y_0.167_z_12.587.dat auto
./evaluate DESY_202107/sam1a/y_0.500_z_12.587.dat auto
./evaluate DESY_202107/sam1a/y_0.167_z_12.589.dat auto
./evaluate DESY_202107/sam1a/y_-0.166_z_12.589.dat auto
./evaluate DESY_202107/sam1a/y_-0.500_z_12.589.dat auto
./evaluate DESY_202107/sam1a/y_-0.166_z_12.591.dat auto
./evaluate DESY_202107/sam1a/y_0.167_z_12.591.dat auto
./evaluate DESY_202107/sam1a/y_0.500_z_12.591.dat auto
./evaluate DESY_202107/sam1a/y_0.167_z_12.593.dat auto
./evaluate DESY_202107/sam1a/y_-0.166_z_12.593.dat auto
./evaluate DESY_202107/sam1a/y_-0.500_z_12.593.dat auto
.

## 2D integration

In [157]:
data = [load_tifs(file) for file in files]

output2d = ais.integrate2d(data, npt_rad=intpoints, npt_azim=3600, correctSolidAngle=True, polarization_factor=0.99)
output1d = ais.integrate1d(data, npt=intpoints, correctSolidAngle=True, polarization_factor=0.99)

In [180]:
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(14,12))
ax1.imshow(np.where(output2d[0]==0, np.nan, output2d[0]), vmin=0, vmax=0.5e11)

ax2.plot(output1d[0], output1d[1])
ax2.set_xlim(np.min(output1d[0]),np.max(output1d[0]))
ax2.set_yscale('log')

for name, pos in zip(peak_name, peak_pos):
    
    if np.min(output1d[0]) < pos < np.max(output1d[0]):

        frac = 1.42*intpoints*(pos-np.min(output1d[0]))/(np.max(output1d[0]))
        
        # draw line and print name
        ax2.axvline(pos, alpha=0.3, c='r')
        ax1.axvline(frac, alpha=0.3, c='r')
        
        ax1.text(frac, 3500, name, horizontalalignment = 'center', c='r')
        ax2.text(pos, np.max(output1d[1]), name, horizontalalignment = 'center', c='r')


plt.show()